In [250]:
import time
import re, string
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as pat
import pandas as pd
import datetime
from collections import Counter
from random import randint

import scipy as sp
import scipy.cluster
import scipy.cluster.hierarchy as hierarchy
import scipy.spatial.distance

from sklearn.cluster import KMeans
import sklearn.datasets as sk_data
import sklearn.metrics as metrics
from sklearn import mixture
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.utils.extmath import randomized_svd

#import matplotlib as mpl
import seaborn as sns

import nltk
#nltk.download()
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
%matplotlib inline

matplotlib_colors = ['k','b', 'g', 'r', 'c', 'm', 'grey', 'y', 'orange', 'pink', 'mediumspringgreen', 'darkcyan', 'darkviolet', 'darkslateblue', 'darkslategrey', 'skyblue', 'lime','indianred','maroon','plum','darkolivegreen','dodgerblue','indianred','lightgray','orchid']

# Generating Training and Test Cases

In [2]:
MA_Data = pd.read_pickle('Massachusetts.pkl')

In [3]:
MA_Data_Shuffle = MA_Data.sample(frac=1)
MA_Data_Shuffle = MA_Data_Shuffle.reset_index()
MA_Data_Shuffle = MA_Data_Shuffle.rename(columns = {'index':'original_index'})
MA_Data_Shuffle

,original_index,State,Station Name (LEA),NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,DEMIL IC,Ship Date
0,230,MA,GLOUCESTER POLICE DEPT,5855-01-228-0939,NIGHT VISION GOGGLE,1,Each,6000.00,F,1.0,2012-01-18
1,182,MA,FAIRHAVEN POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2008-02-26
2,1239,MA,WELLFLEET POLICE DEPT,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",1,Each,138.00,D,1.0,2000-07-25
3,160,MA,EASTON POLICE,5855-01-522-2502,"EYEPIECE ASSEMBLY,OPTICAL INSTRUMENT",1,Each,423.06,D,1.0,2011-11-05
4,365,MA,IPSWICH POLICE DEPT,1005-00-999-1871,"STOCK,GUN,SHOULDER",2,Each,707.36,B,3.0,2013-05-20
5,800,MA,REHOBOTH POLICE,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2006-06-08
6,1215,MA,WATERTOWN POLICE DEPT,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",1,Each,138.00,D,1.0,1999-12-27
7,475,MA,MEDFORD POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2006-02-21
8,700,MA,NEW BEDFORD POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2012-01-17
9,547,MA,MILFORD POLICE DEPT,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",1,Each,138.00,D,1.0,1995-04-11


In [4]:
Master_training = MA_Data_Shuffle.sample(round(len(MA_Data_Shuffle)*2/3), replace=False)
Master_training

,original_index,State,Station Name (LEA),NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,DEMIL IC,Ship Date
674,20,MA,AUBURN POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2012-04-11
507,52,MA,BRISTOL COUNTY SHERIFF DEPT,1240-01-411-1265,"SIGHT,REFLEX",10,Each,315.00,D,1.0,2012-03-14
699,417,MA,LYNN POLICE DEPT,1005-01-128-9936,"RIFLE,5.56 MILLIMETER",1,Each,749.00,D,1.0,2014-11-24
400,19,MA,AUBURN POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2012-04-11
442,1244,MA,WEST BROOKFIELD POLICE DEPT,1005-00-726-5655,"PISTOL,CALIBER .45,AUTOMATIC",1,Each,58.71,D,1.0,2002-04-25
899,867,MA,REVERE POLICE DEPT,5855-00-629-5327,NIGHT VISION SIGHT CREW SERVED WEAPONS,1,Each,3433.00,F,1.0,2011-02-12
123,1195,MA,WAREHAM POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2005-12-28
825,528,MA,METHUEN POLICE DEPT,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",1,Each,138.00,D,1.0,1999-12-27
403,333,MA,IPSWICH POLICE DEPT,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",1,Each,138.00,D,1.0,2007-06-21
104,396,MA,LEXINGTON POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2012-12-03


In [5]:
Master_test = MA_Data_Shuffle.copy()
for orignal_index in Master_training['original_index']:
    Master_test = Master_test[Master_test.original_index != orignal_index]

Master_test

,original_index,State,Station Name (LEA),NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,DEMIL IC,Ship Date
0,230,MA,GLOUCESTER POLICE DEPT,5855-01-228-0939,NIGHT VISION GOGGLE,1,Each,6000.00,F,1.0,2012-01-18
1,182,MA,FAIRHAVEN POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2008-02-26
4,365,MA,IPSWICH POLICE DEPT,1005-00-999-1871,"STOCK,GUN,SHOULDER",2,Each,707.36,B,3.0,2013-05-20
7,475,MA,MEDFORD POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2006-02-21
14,481,MA,MEDFORD POLICE DEPT,1005-00-179-0300,"RIFLE,7.62 MILLIMETER",1,Each,1278.00,D,1.0,1996-06-04
15,358,MA,IPSWICH POLICE DEPT,6260-01-247-0366,"LIGHT,CHEMILUMINESCENT",1,Box,37.15,B,3.0,2014-03-21
17,472,MA,MEDFORD POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2006-02-21
18,1156,MA,WAREHAM POLICE DEPT,5315-01-233-8608,"PIN,SHOULDER,HEADLESS",2,Each,5.87,B,3.0,2013-01-14
20,1159,MA,WAREHAM POLICE DEPT,5305-01-484-7075,"SCREW,CAP,HEXAGON HEAD",10,Each,2.90,B,3.0,2013-02-19
22,1190,MA,WAREHAM POLICE DEPT,5360-01-396-0256,"SPRING,HELICAL,COMPRESSION",10,Each,0.30,B,3.0,2013-10-24


In [6]:
testset_1 = Master_test.sample(round(len(MA_Data_Shuffle)*1/12), replace=False)

In [7]:
testset_4 = Master_test.copy()
for orignal_index in testset_1['original_index']:
    testset_4 = testset_4[testset_4.original_index != orignal_index]

In [8]:
testset_2 = testset_4.sample(round(len(MA_Data_Shuffle)*1/12), replace=False)

In [9]:
for orignal_index in testset_2['original_index']:
    testset_4 = testset_4[testset_4.original_index != orignal_index]

In [10]:
testset_3 = testset_4.sample(round(len(MA_Data_Shuffle)*1/12), replace=False)

In [11]:
for orignal_index in testset_3['original_index']:
    testset_4 = testset_4[testset_4.original_index != orignal_index]

In [12]:
testset_1

,original_index,State,Station Name (LEA),NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,DEMIL IC,Ship Date
405,465,MA,MBTA TRANSIT POLICE,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2002-09-23
628,408,MA,LEYDEN POLICE DEPT,2320-01-128-9551,"TRUCK,UTILITY",1,Each,50458.00,C,1.0,2012-02-06
22,1190,MA,WAREHAM POLICE DEPT,5360-01-396-0256,"SPRING,HELICAL,COMPRESSION",10,Each,0.30,B,3.0,2013-10-24
444,799,MA,REHOBOTH POLICE,2340-DS-ATV-4WHE,"ALL TERRAIN VEHICLE, 4 WHEEL",1,Each,1000.00,B,NaN,2012-11-07
350,753,MA,NORWOOD POLICE DEPT,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",1,Each,138.00,D,1.0,1994-01-31
1279,2,MA,ACTON POLICE DEPT,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",1,Each,138.00,D,1.0,2000-09-06
948,982,MA,SOMERSET POLICE DEPT,8470-01-476-5648,"PAD SET,SUSPENSION SYSTEM,ACH",6,Set,63.66,B,3.0,2011-11-09
529,534,MA,METHUEN POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2008-07-29
1120,337,MA,IPSWICH POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2007-06-21
460,898,MA,ROCKLAND POLICE DEPT,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",1,Each,138.00,D,1.0,2000-07-18


In [13]:
testset_2

,original_index,State,Station Name (LEA),NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,DEMIL IC,Ship Date
1252,764,MA,ORLEANS POLICE DEPT,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",1,Each,138.00,D,1.0,2002-03-19
1029,904,MA,ROCKLAND POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2002-09-09
861,749,MA,NORWOOD POLICE DEPT,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",1,Each,138.00,D,1.0,1994-01-31
1032,907,MA,ROCKLAND POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2002-09-09
485,1076,MA,SWANSEA POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2006-06-10
842,264,MA,GROTON POLICE DEPT,1005-00-856-6885,"RIFLE,5.56 MILLIMETER",1,Each,120.00,D,1.0,2011-10-19
769,684,MA,NEW BEDFORD POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2012-01-17
870,656,MA,MILTON POLICE DEPT,5855-01-468-4169,"ILLUMINATOR,INFRARED",1,Each,776.79,D,1.0,2012-12-20
421,445,MA,MARSHFIELD POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,1998-12-24
608,1108,MA,TYNGSBOROUGH POLICE DEPT,1005-01-128-9936,"RIFLE,5.56 MILLIMETER",1,Each,749.00,D,1.0,2014-12-01


In [14]:
testset_3

,original_index,State,Station Name (LEA),NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,DEMIL IC,Ship Date
413,267,MA,GROTON POLICE DEPT,1005-00-856-6885,"RIFLE,5.56 MILLIMETER",1,Each,120.00,D,1.0,2011-10-19
385,97,MA,COHASSET POLICE DEPT,1005-00-856-6885,"RIFLE,5.56 MILLIMETER",1,Each,120.00,D,1.0,2011-10-18
1161,387,MA,LAWRENCE POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2012-04-26
1070,181,MA,FAIRHAVEN POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2008-02-26
960,744,MA,NORTON POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2012-04-17
505,851,MA,REVERE POLICE DEPT,1005-00-726-5655,"PISTOL,CALIBER .45,AUTOMATIC",1,Each,58.71,D,1.0,2011-12-23
28,1290,MA,WESTPORT POLICE DEPARTMENT,5855-01-447-8992,"ILLUMINATOR,INFRARED",1,Each,1038.00,D,1.0,2011-07-15
278,515,MA,METHUEN POLICE DEPT,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",1,Each,138.00,D,1.0,1999-12-27
823,970,MA,SOMERSET POLICE DEPT,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",1,Each,138.00,D,1.0,2005-08-18
100,715,MA,NEW BEDFORD POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2012-01-17


In [15]:
testset_4

,original_index,State,Station Name (LEA),NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,DEMIL IC,Ship Date
1,182,MA,FAIRHAVEN POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2008-02-26
7,475,MA,MEDFORD POLICE DEPT,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1,Each,499.00,D,1.0,2006-02-21
15,358,MA,IPSWICH POLICE DEPT,6260-01-247-0366,"LIGHT,CHEMILUMINESCENT",1,Box,37.15,B,3.0,2014-03-21
18,1156,MA,WAREHAM POLICE DEPT,5315-01-233-8608,"PIN,SHOULDER,HEADLESS",2,Each,5.87,B,3.0,2013-01-14
25,193,MA,FOXBOROUGH POLICE DEPT,1005-01-128-9936,"RIFLE,5.56 MILLIMETER",1,Each,749.00,D,1.0,2014-09-16
30,986,MA,SOMERSET POLICE DEPT,1005-01-453-5383,"RAIL COVER,9 RIB",30,Package,2.14,B,3.0,2011-11-05
34,879,MA,REVERE POLICE DEPT,1005-00-726-5655,"PISTOL,CALIBER .45,AUTOMATIC",1,Each,58.71,D,1.0,2011-12-23
46,92,MA,COHASSET POLICE DEPT,1005-00-856-6885,"RIFLE,5.56 MILLIMETER",1,Each,120.00,D,1.0,2011-10-18
79,96,MA,COHASSET POLICE DEPT,1005-00-856-6885,"RIFLE,5.56 MILLIMETER",1,Each,120.00,D,1.0,2011-10-18
82,579,MA,MILTON POLICE DEPT,5855-01-468-4169,"ILLUMINATOR,INFRARED",1,Each,776.79,D,1.0,2012-12-20
